<a href="https://colab.research.google.com/github/vianki707/sis25b/blob/main/EJERCICIO1_REAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ejercicio 1. Manipulacion de datos**

In [175]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Configuración
num_empleados = 50
departamentos = ["Ventas", "TI", "Recursos Humanos", "Finanzas", "Operaciones"]
cargos = ["Analista", "Gerente", "Asistente", "Especialista", "Director"]

# 1. Tabla: Empleados (Maestro)
empleados = []
for i in range(1, num_empleados + 1):
    id_emp = f"E{i:03d}"
    nombre = f"Empleado_{i}"
    depto = random.choice(departamentos)
    cargo = random.choice(cargos)
    salario = random.randint(800, 3500)
    fecha_ingreso = datetime(2018, 1, 1) + timedelta(days=random.randint(0, 2000))
    genero = random.choice(["M", "F"])
    empleados.append([id_emp, nombre, depto, cargo, salario, fecha_ingreso.strftime("%Y-%m-%d"), genero])

df_empleados = pd.DataFrame(empleados, columns=["ID_Empleado", "Nombre", "Departamento", "Cargo", "Salario_Mensual", "Fecha_Ingreso", "Genero"])

# 2. Tabla: Evaluaciones de Desempeño (Hechos - Anual)
# Asumimos la evaluación del último año
evaluaciones = []
for emp in empleados:
    id_emp = emp[0]
    # Score 1 to 5
    score = random.choices([1, 2, 3, 4, 5], weights=[5, 10, 40, 35, 10])[0] # Weighted towards 3 and 4
    fecha_eval = "2023-12-15"
    evaluaciones.append([id_emp, fecha_eval, score])

df_evaluaciones = pd.DataFrame(evaluaciones, columns=["ID_Empleado", "Fecha_Evaluacion", "Puntaje_Desempeno"])

# 3. Tabla: Registro de Asistencia (Hechos - Mensual Resumido)
# Datos de los últimos 3 meses para cada empleado
asistencia = []
meses = ["2024-01", "2024-02", "2024-03"]

for emp in empleados:
    id_emp = emp[0]
    for mes in meses:
        dias_ausente = random.choices([0, 1, 2, 3], weights=[70, 15, 10, 5])[0]
        horas_extra = random.randint(0, 10)
        asistencia.append([id_emp, mes, dias_ausente, horas_extra])

df_asistencia = pd.DataFrame(asistencia, columns=["ID_Empleado", "Mes", "Dias_Ausente", "Horas_Extra"])

In [176]:
print("Empleados Head:")
print(df_empleados.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\nEvaluaciones Head:")
print(df_evaluaciones.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\nAsistencia Head:")
print(df_asistencia.head().to_markdown(index=False, numalign="left", stralign="left"))

Empleados Head:
| ID_Empleado   | Nombre     | Departamento     | Cargo        | Salario_Mensual   | Fecha_Ingreso   | Genero   |
|:--------------|:-----------|:-----------------|:-------------|:------------------|:----------------|:---------|
| E001          | Empleado_1 | Recursos Humanos | Gerente      | 1849              | 2020-12-14      | F        |
| E002          | Empleado_2 | Ventas           | Asistente    | 1660              | 2022-03-06      | F        |
| E003          | Empleado_3 | Recursos Humanos | Asistente    | 1810              | 2019-04-15      | F        |
| E004          | Empleado_4 | Ventas           | Analista     | 2110              | 2021-01-27      | F        |
| E005          | Empleado_5 | Operaciones      | Especialista | 1805              | 2018-01-24      | M        |

Evaluaciones Head:
| ID_Empleado   | Fecha_Evaluacion   | Puntaje_Desempeno   |
|:--------------|:-------------------|:--------------------|
| E001          | 2023-12-15         | 4    

In [177]:
n_by_state = df_asistencia.groupby("Dias_Ausente")["ID_Empleado"].count()
print(n_by_state)


Dias_Ausente
0    97
1    20
2    25
3     8
Name: ID_Empleado, dtype: int64


Union de los datos en las tablas

In [178]:
df_tabla_conjunta2 = pd.merge(df_empleados,df_asistencia,on="ID_Empleado",how="inner")

print(df_tabla_conjunta2)


    ID_Empleado       Nombre      Departamento      Cargo  Salario_Mensual  \
0          E001   Empleado_1  Recursos Humanos    Gerente             1849   
1          E001   Empleado_1  Recursos Humanos    Gerente             1849   
2          E001   Empleado_1  Recursos Humanos    Gerente             1849   
3          E002   Empleado_2            Ventas  Asistente             1660   
4          E002   Empleado_2            Ventas  Asistente             1660   
..          ...          ...               ...        ...              ...   
145        E049  Empleado_49          Finanzas   Analista              861   
146        E049  Empleado_49          Finanzas   Analista              861   
147        E050  Empleado_50            Ventas   Director             3421   
148        E050  Empleado_50            Ventas   Director             3421   
149        E050  Empleado_50            Ventas   Director             3421   

    Fecha_Ingreso Genero      Mes  Dias_Ausente  Horas_Extra  


In [179]:
#Prom. dias ausencia
prom_ausencias = (df_asistencia.groupby("ID_Empleado")["Dias_Ausente"].mean().reset_index().rename(columns={"Dias_Ausente": "Promedio_Dias_Ausente"}))

#Prom. horas extra
prom_horas_extra = (df_asistencia.groupby("ID_Empleado")["Horas_Extra"].mean().reset_index())

#Metodo Merge
df_base = pd.merge(df_empleados,df_evaluaciones,on="ID_Empleado",how="inner")
df_base = pd.merge(df_base,prom_ausencias,on="ID_Empleado",how="inner")
df_base = pd.merge(df_base,prom_horas_extra,on="ID_Empleado",how="inner")

df_base.head()


,ID_Empleado,Nombre,Departamento,Cargo,Salario_Mensual,Fecha_Ingreso,Genero,Fecha_Evaluacion,Puntaje_Desempeno,Promedio_Dias_Ausente,Horas_Extra
0,E001,Empleado_1,Recursos Humanos,Gerente,1849,2020-12-14,F,2023-12-15,4,0.333333,7.666667
1,E002,Empleado_2,Ventas,Asistente,1660,2022-03-06,F,2023-12-15,3,0.666667,1.666667
2,E003,Empleado_3,Recursos Humanos,Asistente,1810,2019-04-15,F,2023-12-15,4,0.333333,7.333333
3,E004,Empleado_4,Ventas,Analista,2110,2021-01-27,F,2023-12-15,3,0.000000,5.000000
4,E005,Empleado_5,Operaciones,Especialista,1805,2018-01-24,M,2023-12-15,3,0.000000,6.000000


**Pregunta 1: ¿Cuántos empleados cumplen esto? ¿Quiénes son?**

In [180]:
empleados_estrella = df_base[(df_base["Puntaje_Desempeno"]>=4)&(df_base["Promedio_Dias_Ausente"]<1)]
empleados_estrella[["ID_Empleado", "Nombre", "Departamento","Puntaje_Desempeno", "Promedio_Dias_Ausente"]]


,ID_Empleado,Nombre,Departamento,Puntaje_Desempeno,Promedio_Dias_Ausente
0,E001,Empleado_1,Recursos Humanos,4,0.333333
2,E003,Empleado_3,Recursos Humanos,4,0.333333
5,E006,Empleado_6,Recursos Humanos,5,0.333333
17,E018,Empleado_18,Ventas,4,0.000000
20,E021,Empleado_21,Recursos Humanos,4,0.000000
24,E025,Empleado_25,Ventas,4,0.000000
26,E027,Empleado_27,Operaciones,4,0.666667
27,E028,Empleado_28,TI,4,0.333333
32,E033,Empleado_33,Recursos Humanos,5,0.000000
39,E040,Empleado_40,TI,4,0.000000


In [181]:
empleados_estrella = df_base[(df_base["Puntaje_Desempeno"]>=4)&(df_base["Promedio_Dias_Ausente"]<1)]
empleados_estrella[["ID_Empleado", "Nombre", "Departamento","Puntaje_Desempeno", "Promedio_Dias_Ausente"]]

#Cantidad total de empleados estrella
cantidad_estrella_df = empleados_estrella[["ID_Empleado"]].count()
print("Cantidad de empleados estrella:")
print(cantidad_estrella_df)



Cantidad de empleados estrella:
ID_Empleado    12
dtype: int64


* La cantidad de empleados que cumplen con esto son los que se ven en la parte superior como ID_Empleado, en esta ejecucion es de 18, ademas, se pueden visualizar en la tabla superior.

**Pregunta 2: ¿Existe algún empleado con salario alto (ej. > $2000) que tenga bajo desempeño?**

In [182]:
df_salarios = df_base.sort_values(by="Salario_Mensual",ascending=False)
empleados_incoherentes = df_salarios[(df_salarios["Puntaje_Desempeno"]<=2)&(df_salarios["Salario_Mensual"]>2000)]
empleados_incoherentes[["ID_Empleado", "Nombre", "Salario_Mensual", "Puntaje_Desempeno"]]


,ID_Empleado,Nombre,Salario_Mensual,Puntaje_Desempeno
44,E045,Empleado_45,3287,1
36,E037,Empleado_37,2919,2
9,E010,Empleado_10,2602,2
38,E039,Empleado_39,2180,2
16,E017,Empleado_17,2094,2


* Si existen, esto se confirma ya que en la tabla superior se presentan empleados que cumplen con los requisitos que nos permiten determinar eso.